# Chapter 5 - Resampling Methods

- [Lab 5.3.1 The Validation Set Approach](#lab-5.3.1)
- [Lab 5.3.2 Leave-One-Out Cross-Validation](#lab-5.3.2)
- [Lab 5.3.3 k-Fold Cross-Validation](#lab-5.3.3)
- [Lab 5.3.4 The Bootstrap](#lab-5.3.4)

### Imports and Configurations

In [1]:
# Standard imports
import warnings

# Use rpy2 for loading R datasets
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import data as rdata
from rpy2.robjects import pandas2ri

# Math and data processing
import numpy as np
import scipy as sp
import pandas as pd

# scikit-learn
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.utils import resample
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Visulization
from IPython.display import display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.style.use('ggplot')

<a id='lab-5.3.1'></a>

### Lab 5.3.1 The Validation Set Approach

In [2]:
# Auto dataset is in R ISLR package
islr = importr('ISLR')
auto_rdf = rdata(islr).fetch('Auto')['Auto']
auto = pandas2ri.ri2py(auto_rdf)
display(auto)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina
10,15.0,8.0,390.0,190.0,3850.0,8.5,70.0,1.0,amc ambassador dpl


In [3]:
# Simple linear regression features and response
features = ['horsepower']
response = ['mpg']
X = auto[features]
y = auto[response]

# Split Auto data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=196, random_state=47)

# Regression
auto_slr = LinearRegression()
auto_slr.fit(X_train, y_train)

# Prediction and MSE
y_pred = auto_slr.predict(X_test)
print("SLR MSE = ", mean_squared_error(y_test, y_pred))

SLR MSE =  25.273723993


In [4]:
# Polynomial regression features of degree 2
poly2 = PolynomialFeatures(degree=2)
X2 = poly2.fit_transform(X)

# Split Auto data into train and test sets
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size=196, random_state=47)

# Regression
auto_poly2 = LinearRegression()
auto_poly2.fit(X2_train, y_train)

# Prediction and MSE
y2_pred = auto_poly2.predict(X2_test)
print("Polynomial regression of degree 2: MSE = ", mean_squared_error(y_test, y2_pred))

Polynomial regression of degree 2: MSE =  18.8690031195


In [5]:
# Polynomial regression features of degree 3
poly3 = PolynomialFeatures(degree=3)
X3 = poly3.fit_transform(X)

# Split Auto data into train and test sets
X3_train, X3_test, y_train, y_test = train_test_split(X3, y, test_size=196, random_state=47)

# Regression
auto_poly3 = LinearRegression()
auto_poly3.fit(X3_train, y_train)

# Prediction and MSE
y3_pred = auto_poly3.predict(X3_test)
print("Polynomial regression of degree 3: MSE = ", mean_squared_error(y_test, y3_pred))

Polynomial regression of degree 3: MSE =  18.8333669959


<a id='lab-5.3.2'></a>

### 5.3.2 Leave-One-Out Cross-Validation

In [6]:
# Polynomial regression over degrees from 1 (simple linear) to 5
auto_poly = LinearRegression()
loocv = LeaveOneOut()

for poly_deg in range(1, 6):
    print("\nPolynomial regression of degree {}:".format(poly_deg))
    poly = PolynomialFeatures(degree=poly_deg)
    X_d = poly.fit_transform(X)
    scores = cross_val_score(auto_poly, X_d, y, cv=loocv, scoring='neg_mean_squared_error')
    loocv_mse = scores.mean() * (-1)  # sign-flip to convert score to MSE
    print('  MSE = {}\n'.format(loocv_mse))


Polynomial regression of degree 1:
  MSE = 24.231513517929226


Polynomial regression of degree 2:
  MSE = 19.24821312448939


Polynomial regression of degree 3:
  MSE = 19.334984064114092


Polynomial regression of degree 4:
  MSE = 19.42443030854574


Polynomial regression of degree 5:
  MSE = 19.033219754727583



<a id='lab-5.3.3'></a>

### 5.3.3 k-Fold Cross-Validation

In [7]:
# Polynomial regression over degrees from 1 (simple linear) to 10
auto_poly = LinearRegression()
kfold = KFold(n_splits=10, random_state=47)

for poly_deg in range(1, 11):
    print("\nPolynomial regression of degree {}:".format(poly_deg))
    poly = PolynomialFeatures(degree=poly_deg)
    X_d = poly.fit_transform(X)
    scores = cross_val_score(auto_poly, X_d, y, cv=kfold, scoring='neg_mean_squared_error')
    loocv_mse = scores.mean() * (-1)  # sign-flip to convert score to MSE
    print('  MSE = {}\n'.format(loocv_mse))


Polynomial regression of degree 1:
  MSE = 27.439933652339857


Polynomial regression of degree 2:
  MSE = 21.235840055802118


Polynomial regression of degree 3:
  MSE = 21.3366061833284


Polynomial regression of degree 4:
  MSE = 21.353886987563506


Polynomial regression of degree 5:
  MSE = 20.905633737044845


Polynomial regression of degree 6:
  MSE = 20.782704427497574


Polynomial regression of degree 7:
  MSE = 20.953103378424892


Polynomial regression of degree 8:
  MSE = 21.07713162886134


Polynomial regression of degree 9:
  MSE = 21.036781313639857


Polynomial regression of degree 10:
  MSE = 20.98095645636944



<a id='lab-5.3.4'></a>

### 5.3.4 The Bootstrap

In [8]:
# Auto dataset is in R ISLR package
islr = importr('ISLR')
portfolio_rdf = rdata(islr).fetch('Portfolio')['Portfolio']
portfolio = pandas2ri.ri2py(portfolio_rdf)
display(portfolio)

,X,Y
1,-0.895251,-0.234924
2,-1.562454,-0.885176
3,-0.417090,0.271888
4,1.044356,-0.734198
5,-0.315568,0.841983
6,-1.737124,-2.037191
7,1.966413,1.452957
8,2.152868,-0.434139
9,-0.081208,1.450809
10,-0.891782,0.821016


In [9]:
# Function to calculate the alpha for portofolio allocation
def alpha(data):
    """
    data: pandas dataframe with two columns X and Y.
    """

    sigma = data.cov()  # covariance matrix
    var_x = sigma.X['X']
    var_y = sigma.Y['Y']
    cov_xy = sigma.X['Y']
    alpha = (var_y - cov_xy) / (var_x + var_y - 2 * cov_xy)
    return alpha
alpha_original = alpha(portfolio)
print("Portfolio alpha = ", alpha_original)

Portfolio alpha =  0.575832074593


In [10]:
# Bootstrap with B=1000 on portfolio data
N = portfolio.shape[0]
B = 1000
portfolio_b = resample(portfolio, n_samples=N*B, random_state=42)
alphas = [alpha(group) for name, group in portfolio_b.groupby(np.arange(N * B) // N)]
alpha_bias = np.mean(alphas) - alpha_original
alpha_se = np.std(alphas)
alpha_bootstrap = pd.DataFrame([[alpha_original, alpha_bias, alpha_se],],
                              columns=['original', 'bias', 'std. error'], index=['alpha'])
display(alpha_bootstrap)

,original,bias,std. error
alpha,0.575832,0.001963,0.089929


In [11]:
# Function to get simple linear regression coefficients for Auto data set
def auto_coef(data, features, response):
    """
    data: pandas dataframe sampled from the Auto data set
    features: a string list of feature names
    response: a string of response names
    """

    auto_reg = LinearRegression()
    auto_reg.fit(data[features], data[response])
    return [auto_reg.intercept_] + list(auto_reg.coef_)

features = ['horsepower']
response = 'mpg'
coef_original = pd.DataFrame([auto_coef(auto, features, response)], columns=['Intercept'] + features, index=[''])
print("\nmpg ~ horsepower coefficients:\n\n", coef_original)


mpg ~ horsepower coefficients:

   Intercept  horsepower
  39.935861   -0.157845


In [12]:
# Bootstrap with B=1000 on Auto data
N = auto.shape[0]
B = 1000
auto_b = resample(auto, n_samples=N*B, random_state=42)
coefs = [auto_coef(group, features, response) for name, group in auto_b.groupby(np.arange(N * B) // N)]
coefs_df = pd.DataFrame(coefs, columns=['Intercept'] + features)
coef_bias = coefs_df.mean() - coef_original
coef_se = coefs_df.std()
coef_bootstrap = pd.concat([coef_original.T.copy(), coef_bias.T, coef_se], axis=1)
coef_bootstrap.columns = ['original', 'bias', 'std. error']
display(coef_bootstrap)

,original,bias,std. error
Intercept,39.935861,0.033521,0.869087
horsepower,-0.157845,-0.000500,0.007503
